In [27]:
import pandas as pd
import urllib.request
import zipfile

In [28]:
# Data Sources
urls = ['https://www.bls.gov/oes/special.requests/oesm17all.zip', 
'https://www.bls.gov/oes/special.requests/oesm16all.zip',
'https://www.bls.gov/oes/special.requests/oesm15all.zip',
'https://www.bls.gov/oes/special.requests/oesm14all.zip',
'https://www.bls.gov/oes/special.requests/oesm13all.zip',
'https://www.bls.gov/oes/special.requests/oesm12all.zip',
'https://www.bls.gov/oes/special.requests/oesm11all.zip',
'https://www.bls.gov/oes/special.requests/oesm10ma.zip',
'https://www.bls.gov/oes/special.requests/oesm09ma.zip',
'https://www.bls.gov/oes/special.requests/oesm08ma.zip']

files = ['data/oesm17all.zip', 
'data/oesm16all.zip',
'data/oesm15all.zip',
'data/oesm14all.zip',
'data/oesm13all.zip',
'data/oesm12all.zip',
'data/oesm11all.zip',
'data/oesm10ma.zip',
'data/oesm09ma.zip',
'data/oesm08ma.zip']

In [30]:
def getData(urls, files):
    for i, j in zip(urls, files):
        urllib.request.urlretrieve(i, j)
        with zipfile.ZipFile(j) as oeszip:
            oeszip.extractall(path = 'data/')            
#getData(urls, files)    

In [31]:
def process2008():
    # Obtain Data
    file200801 = "./Data/MSA__M2008_dl_1.xls"
    file200802 = "./Data/MSA_M2008_dl_2.xls"
    file200803 = "./Data/MSA_M2008_dl_3.xls"

    x200801 = pd.ExcelFile(file200801)
    x200802 = pd.ExcelFile(file200802)
    x200803 = pd.ExcelFile(file200803)

    df200801 = x200801.parse('MSA_dl_1')
    df200802 = x200802.parse('MSA_dl_2')
    df200803 = x200803.parse('MSA_dl_3')

    df2008 = [df200801, df200802, df200803]
    df2008 = pd.concat(df2008)
    return df2008
df2008 = process2008()

In [32]:
df2008

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,NaN,164800,1.9,23.10,48050,...,19.09,29.63,40.65,19570,26400,39710,61640,84550,NaN,NaN
1,AK,11260,"Anchorage, AK",11-0000,Management occupations,major,12430,2.7,40.37,83960,...,36.21,49.31,67.75,38180,52600,75310,102560,140920,NaN,NaN
2,AK,11260,"Anchorage, AK",11-1011,Chief executives,NaN,850,6.1,70.24,146100,...,65.12,#,#,71080,96170,135450,#,#,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and operations managers,NaN,4560,4.8,39.57,82300,...,35.75,48.55,66.62,37100,50860,74360,100980,138570,NaN,NaN
4,AK,11260,"Anchorage, AK",11-1031,Legislators,NaN,40,7.8,*,36710,...,*,*,*,15310,22700,32980,47060,58590,True,NaN
5,AK,11260,"Anchorage, AK",11-2011,Advertising and promotions managers,NaN,80,23.7,33.27,69200,...,28.46,32.61,50.60,42990,48130,59190,67840,105250,NaN,NaN
6,AK,11260,"Anchorage, AK",11-2021,Marketing managers,NaN,220,11.1,39.80,82780,...,32.69,46.79,76.29,38340,50680,68000,97310,158670,NaN,NaN
7,AK,11260,"Anchorage, AK",11-2022,Sales managers,NaN,520,8.9,33.90,70510,...,28.70,40.49,60.26,34160,41800,59700,84220,125350,NaN,NaN
8,AK,11260,"Anchorage, AK",11-2031,Public relations managers,NaN,90,16.2,36.18,75260,...,29.82,41.20,63.59,40370,49850,62030,85690,132260,NaN,NaN
9,AK,11260,"Anchorage, AK",11-3011,Administrative services managers,NaN,780,8.9,30.15,62710,...,29.16,37.78,44.05,34630,45310,60640,78580,91620,NaN,NaN
